Based on "https://github.com/keras-rl/keras-rl/blob/master/examples/dqn_atari.py"

In [1]:
import random

import argparse

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.


## Environment

In [18]:
env = gym.make('MsPacman-v0')
obsertation = env.reset()
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
nb_actions

9

In [3]:
state = env.reset()
state.shape

(210, 160, 3)

In [4]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## Model

### Hyperparameters

In [6]:
K.image_dim_ordering()

'tf'

### Deep Q-learning model

In [15]:
# The model that was described by Mnih et al. (2015).

model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Conv2D(32, (8, 8), strides=(4, 4),activation='relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2),activation='relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1),activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 4617      
Total para

In [8]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=10000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

In [9]:
# Select a policy. We use eps-greedy action selection, which means that a random action is selected
# with probability eps. We anneal eps from 1.0 to 0.1 over the course of 1M steps. This is done so that
# the agent initially explores the environment (high eps) and then gradually sticks to what it knows
# (low eps). We also set a dedicated eps value that is used during testing. Note that we set it to 0.05
# so that the agent still performs some random actions. This ensures that the agent cannot get stuck.
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000)

In [17]:
#policy = BoltzmannGumbelQPolicy()


In [11]:
# The trade-off between exploration and exploitation is difficult and an on-going research topic.
# If you want, you can experiment with the parameters or use a different policy. Another popular one
# is Boltzmann-style exploration:
# policy = BoltzmannQPolicy(tau=1.)
# Feel free to give it a try!

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

In [12]:
# Okay, now it's time to learn something! We capture the interrupt exception so that training
# can be prematurely aborted. Notice that you can the built-in Keras callbacks!
env_name = 'MsPacman-v0'

weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=2500)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env, callbacks=callbacks, nb_steps=1750, log_interval=10000)

# After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)

# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)


Training for 1750 steps ...
Interval 1 (0 steps performed)
 1750/10000 [====>.........................] - ETA: 1:47 - reward: 0.0566done, took 22.853 seconds
Testing for 10 episodes ...
Episode 1: reward: 32.000, steps: 846
Episode 2: reward: 42.000, steps: 930
Episode 3: reward: 35.000, steps: 878
Episode 4: reward: 37.000, steps: 791
Episode 5: reward: 31.000, steps: 880
Episode 6: reward: 35.000, steps: 809
Episode 7: reward: 24.000, steps: 725
Episode 8: reward: 34.000, steps: 852
Episode 9: reward: 37.000, steps: 776
Episode 10: reward: 20.000, steps: 500


In [14]:
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)

dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 27.000, steps: 777
Episode 2: reward: 21.000, steps: 985
Episode 3: reward: 30.000, steps: 799
Episode 4: reward: 7.000, steps: 486
Episode 5: reward: 29.000, steps: 774
Episode 6: reward: 24.000, steps: 721
Episode 7: reward: 33.000, steps: 881
Episode 8: reward: 24.000, steps: 706
Episode 9: reward: 37.000, steps: 759
Episode 10: reward: 24.000, steps: 725
